(c) Juan Gomez 2019. Thanks to Universidad EAFIT for support. This material is part of the course Introduction to Finite Element Analysis

# Convergence of analysis results

## Introduction

In this section we address the problem of convergency of analysis results. We will approach the problem in a loose way proceeding from an engineering point of view. For a thorough discussion the reader is referred to textbooks of numerical analysis, see for instance Systèmes, D. (2012). Particularly we will review the fundamental aspects that must be satisfied by a finite element solution. In the first part we address the problem from the element point of view, while in the final part we study the convergence of particular problem in terms of several self-contained meshes.**After completing this notebook you should be able to:**

* Recognize the total potential energy of a model like the fundamental descriptor of the response.

* Perform a convergency analysis for a given finite element model.

## Definition of convergence

Let $\Pi$ be the total potential energy of a solid in equilibrium and under a elastic stress state. The value of $\Pi$ for a given finite element mesh is given by:

$$
{\Pi _{FE}} =  - \frac{1}{2}{U^T}KU
$$

where $K$ and $U$ are the global stiffness matrix and global nodal displacements vector. If $q$ represents the number of finite elements in a given discretization then we define convergency as the condition that:

$$
\mathop {\lim }\limits_{q \to \infty } {\Pi_{FE}} \to \Pi 
$$



To measure the state of stress associated to a given mesh we need to define a norm. Consider the square energy norm of the error $\left\| {{{\vec e}_h}} \right\|_E^2$. This measure satisfies

$$\left\| {{{\vec e}_h}} \right\|_E^2 > 0$$

and will be used as an error estimate of the accuraccy of the finite element solution. Particularly we will use the following relationship (see Systèmes, D. 2012).

$$
\left\| {{{\vec e}_h}} \right\| \le \alpha {h^k}
$$


from which we can write:


$$
\log \left\| {{{\vec e}_h}} \right\| \approx \log \alpha  + k\log h
$$


In the expression above $k$ is the order of the complete interpolation polynomial present in the mesh and gives a measure of the order of convergence in the finite element solution, while the rate of convergency is given by $\alpha$.


## How to conduct the convergence analysis?

In order to conduct the convergency study we perform a series of finite element analysis. For each mesh we compute $\left\| {\vec u - {{\vec u}_h}} \right\|$ which is equivalent to $\vec e_h$ and where $\vec u$ is the exact solution. In order to find the exact solution we assume that the most refined results have functionals corresponding to ${\Pi _{n - 2}}$, ${\Pi _{n - 1}}$ and ${\Pi _{n}}$ from which:

$$
{\Pi _{Exa}} = \frac{{\Pi _{n - 1}^2 - {\Pi _n}{\Pi _{n - 2}}}}{{(2{\Pi _{n - 1}} - {\Pi _n} - {\Pi _{n - 2}})}}
$$


The procedure is summarized below:


* Solve a series of meshes with solutions given by $\vec u_1, \vec u_2,...,\vec u_n$. Each mesh has a characteristic element size $h$. Here the subscript represents a given discretization.
* For each mesh find the total potential energy:

$${\Pi_h} =  - \frac{1}{2}{U^T}KU$$

* Using the most refined meshes compute the potential energy for the exact solution $\Pi _{Exa}$.

* For each mesh compute:

$$
\frac{{\left\| {{{\vec u}_{Exa}} - {{\vec u}_h}} \right\|}}{{\left\| {{{\vec u}_{Exa}}} \right\|}} \equiv {\left[ {\frac{{{\Pi _{Exa}} - {\Pi _h}}}{{{\Pi _{Exa}}}}} \right]^{1/2}}
$$

and keep track of the results.



* Plot the values of $\log \left( {\frac{{\left\| {{{\vec u}_{Exa}} - {{\vec u}_h}} \right\|}}{{\left\| {{{\vec u}_{Exa}}} \right\|}}} \right)$ vs $\log h$ and determine the slope which upon convergence must be close to the order of the complete polynomial used in the discretization.

### Example: Tappered bar in compression

The figure below shows a tappered bar under a compressive uniformly distributed load of total magnitude $P=0.5$. The bar is of length $l=10$ and its large and short ends given by $h_1 = 2.0$ and $h_2 =0.5$ respectively. The material properties correspond to a Poisson's ratio and Young modulos $\nu=0.30$ and $E=1.0$ respectively. We want to find the converged solution for the bar after using 3-noded triangular elements.





<center><img src="img/tappered.png" alt="files" style="width:500px"></center>

Analyses were conduted with 4 consecutive meshes with decreasing element size given by $h=[1.0, 0.5, 0.25, 0.125]$.


<center><img src="img/tapdata.png" alt="files" style="width:500px"></center>

Now assuming we have consecutive meshes each one obtained after halving the elements in the previous mesh we have the following approximation for the exact total potential energy of the system, computed the last most refined meshes:

$${\Pi _{Exa}} = \frac{{{{1.161}^2} - ( - 1.160)( - 1.155)}}{{ - 2(1.161) - ( - 1.160) - ( - 1.155)}} = -1.160.$$

To compute the energy norm of the error we use:

$$\frac{{\left\| {{{\vec u}_{Exa}} - {{\vec u}_{FE}}} \right\|}}{{\left\| {{{\vec u}_{Exa}}} \right\|}} = {\left[ {\frac{{{\Pi _{Exa}} - {\prod _{FE}}}}{{{\Pi _{Exa}}}}} \right]^{1/2}}.$$



To measure convergence we plot:

$$\log \left( {\left\| {{{\vec u}_{Exa}} - {{\vec u}_{FE}}} \right\|} \right) = \log c + k\log h$$


<center><img src="img/convrate.png" alt="files" style="width:500px"></center>

### Computation of potential energy

The files **Tnodes.txt**, **Teles.txt** , **Tmater.txt** and **Tloads.txt** contained in the **files** folder correspond to the finite element mesh for one of the **tappred bar** model. To compute the potential energy

$${\Pi_h} =  - \frac{1}{2}{U^T}KU$$

for the discretization the model is solved first using the dense assembly option.

In [1]:
%matplotlib inline        
import matplotlib.pyplot as plt
import numpy as np
import sympy as sym
import solidspy.assemutil as ass
import solidspy.postprocesor as pos
import solidspy.solutil as sol

In [2]:
def readin():
    nodes    = np.loadtxt('files/' + 'Tnodes.txt', ndmin=2)
    mats     = np.loadtxt('files/' + 'Tmater.txt', ndmin=2)
    elements = np.loadtxt('files/' + 'Teles.txt', ndmin=2, dtype=np.int)
    loads    = np.loadtxt('files/' + 'Tloads.txt', ndmin=2)

    return nodes, mats, elements, loads
nodes, mats, elements, loads = readin()

In [3]:
DME , IBC , neq = ass.DME(nodes, elements)
KG = ass.assembler(elements, mats, nodes, neq, DME, sparse=False)
RHSG = ass.loadasem(loads, IBC, neq)
UG = sol.static_sol(KG, RHSG)
UC = pos.complete_disp(IBC, nodes, UG)
Energy = np.dot(np.dot(UG.T , KG) , UG)
print(Energy)

0.5785646424764832


### Glossary of terms

**Converged solution:** A finite element solution associated to a specific discretization and numerically considered as close enough to the exact solution.

**Norm:** Specific mathematical form of measuring the magnitude of a vector in a given basis.

**Convergency analysis:** A series of finite element analysis of a given computational domain with increasingly refined meshes in order to determined an accepatble solution.

## Class activity.

Consider the cantilever beam shown below:


<center><img src="img/canti.png" alt="files" style="width:500px"></center>


with analytic solution (Timoshenko and Goodier,1976) ) given by:

$$u =  - \frac{P}{{2EI}}{x^2}y - \frac{{\nu P}}{{6EI}}{y^3} + \frac{P}{{2IG}}{y^3} + \left( {\frac{{P{l^2}}}{{2EI}} - \frac{{P{c^2}}}{{2IG}}} \right)y$$

$$v = \frac{{\nu P}}{{2EI}}x{y^2} + \frac{P}{{6EI}}{x^3} - \frac{{P{l^2}}}{{2EI}}x + \frac{{P{l^3}}}{{3EI}}$$

$${\varepsilon _{xx}} = \frac{{\partial u}}{{\partial x}} \equiv  - \frac{P}{{EI}}xy$$

$${\varepsilon _{yy}} = \frac{{\partial v}}{{\partial y}} \equiv \frac{{\nu P}}{{EI}}xy$$

$${\gamma _{xy}} = \frac{{\partial u}}{{\partial y}} + \frac{{\partial v}}{{\partial x}} \equiv \frac{P}{{2IG}}\left( {{y^2} - {c^2}} \right).$$

The values of the material parameters are $E=1000.0$ and $\nu=0.30$ while the beam dimensions are $l=24$ and $2c=8$ and the applied load is $P=50$ (upwards).

Perform a finite element simulation using a series of refined meshes with charateristic element size corresponding to $h=[6.0,3.0,1.5,0.75,0.375]$ and show that convergence is achieved. To conduct the finite element analysis fill out \cref{problem}



### References

* Bathe, Klaus-Jürgen. (2006) Finite element procedures. Klaus-Jurgen Bathe. Prentice Hall International.

* Juan Gómez, Nicolás Guarín-Zapata (2018). SolidsPy: 2D-Finite Element Analysis with Python, <https://github.com/AppliedMechanics-EAFIT/SolidsPy>.

* Systèmes, D. (2012). ABAQUS 6.12 Theory manual. Dassault Systèmes Simulia Corp., Providence, Rhode Island.

* Timoshenko, S.P., and Goodier, J.N. (1976). Theory of Elasticity. International Student Edition. McGraw-Hill International.

In [1]:
from IPython.core.display import HTML
def css_styling():
    styles = open('./nb_style.css', 'r').read()
    return HTML(styles)
css_styling()